<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/path_finder_agent_ida_star_genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import random

In [7]:
heuristic_distance = {
                        'New Delhi': 4,
                        'A': 5,
                        'B': 2,
                        'C': 10,
                        'D': 5,
                        'E': 3,
                        'Chennai': 0

                        }

In [8]:

class Node:
    def __init__(self,key,value,parent=None):
        self.key=key
        self.value=value
        self.parent=None
        self.idx=0

    def priority(self):
        return self.key+heuristic_distance[self.value]
    def set_parent(self,parent):
        self.parent=parent
    def __str__(self):
        return 'Value={}, Key={}'.format(self.value,self.key)
class PriorityQueue:
    def __init__(self):
        self.queue=[None]
        self.size=0
    def parent(self,i):
        return int(i/2)
    def left_child(self,i):
        return 2*i
    def right_child(self,i):
        return 2*i+1
    def exchange(self,i,j):
        self.queue[i],self.queue[j]=self.queue[j],self.queue[i]
        self.queue[i].idx,self.queue[j].idx=self.queue[j].idx,self.queue[i].idx
    def less(self,i,j):
        return self.queue[i].priority() < self.queue[j].priority()
    def more(self,i,j):
        return self.queue[i].priority() > self.queue[j].priority()
    def swim(self,k):
        if self.size ==1:
           return
        while(k > 1 and self.more(self.parent(k),k)):
             self.exchange(k,self.parent(k))
             k=self.parent(k)
    def sink(self,k):
        while self.left_child(k) <= self.size:
              left_child_idx= self.left_child(k)
              idx=left_child_idx
              right_child_idx= self.right_child(k)

              if right_child_idx <= self.size and self.more(left_child_idx,right_child_idx):
                 idx=right_child_idx
              if self.more(k,idx):
                 self.exchange(k,idx)
              k=idx

    def is_empty(self):
        return self.size==0
    def resize(self,new_size):
        if new_size >self.size:
           queue_resized=[None]*(new_size)
           for  i in range(self.size+1):
                queue_resized[i]=self.queue[i]
           self.queue=queue_resized

    def min(self):
        if self.is_empty() == False:
            return self.queue[1]
        return None
    def show(self):
         print('Priority Queue size is {}'.format(self.size))
         for i in range(self.size+1):
            if i >0:
                print(self.queue[i])

    def insert(self,node):
        if self.size == (len(self.queue)-1):
            self.resize(2*len(self.queue))

        self.size=self.size+1
        #print('Inserting node {} '.format(node))
        node.idx=self.size
        self.queue[self.size]=node


        self.swim(self.size)

    def extract_min(self):
        if self.is_empty() == False:
            min=self.min()
            self.exchange(1,self.size)
            self.size=self.size-1
            self.sink(1)
            if self.is_empty() == True:
               self.queue=[None]

            return min

        return None
    def contains_value(self,value):
        for i in range(1,self.size+1):
            if self.queue[i].value == value:
                return True
        return False

In [55]:

class Graph:

     def build_graph(self):
        all_nodes ={}
        self.graph={}
        for key_str in self.graph_str:
            neighbours=[]
            for (m,(weight,_)) in self.graph_str[key_str]:
                neighbour=None
                if m not in all_nodes.keys():
                    neighbour=Node(0,m)
                    all_nodes[m]=neighbour
                else:
                    neighbour=all_nodes[m]
                neighbours.append((neighbour,weight))
            self.graph[key_str]=neighbours

     def get_neighbors(self,v):
         if v in self.graph:
            return self.graph[v]
         else:
            return None
     def get_random_path(self,start_node,stop_node):
        random_path=[start_node.value]
        while random_path[-1] != stop_node.value:
            neighbours=self.get_neighbors(random_path[-1])
            next_random_node = random.choice(neighbours)[0]
            if next_random_node.value not in random_path:
               random_path.append(next_random_node.value)
        return random_path
     def get_weight(self,first_node_str,second_node_str):
         #print('first node  {} , second node {}'.format(first_node_str,second_node_str))
         edges=self.graph_str[first_node_str]
         for edge in edges:
            if edge[0] == second_node_str:
               return edge[1][0]
         return float('inf')

     def aStarSearch(self,start_node, stop_node):
         open_set=PriorityQueue()
         closed_set=set()
         open_set.insert(start_node)
         while open_set.is_empty() == False:
               n=open_set.extract_min()
               if n == None:
                  print('Path does not exist!')
                  return None
               if n.value == stop_node.value:
                    path = []
                    stop_node_clone=n
                    while n.parent is not None:
                        path.append(n.value)
                        n = n.parent
                    path.append(start_node.value)
                    path.reverse()
                    print('Cost is {}'.format(stop_node_clone.key))
                    return path
               neighbours=graph.get_neighbors(n.value)
               #print('Min node is {}'.format(n.value))
               for (m,weight) in neighbours:
                    if m.value not in closed_set and open_set.contains_value(m.value) == False:
                       g_m=n.key+weight
                       m.key=g_m
                       m.parent=n
                       #print('Inserting node is {} '.format(m))
                       open_set.insert(m)

                    else:
                        if m.key > n.key+weight:
                           m.key=n.key+weight
                           m.parent=n
                           #print('Swimming up {} with cost {}'.format(m.idx,m.key))
                           #print('less cost {} for {}'.format(m.key,m))
                           open_set.swim(m.idx)
                           if m.value in closed_set:
                              closed_set.remove(m.value)
                              #print('Inserting node is {} due to less cost'.format(n))
                              open_set.insert(m)


               closed_set.add(n.value)
         return None
     def ida_star_search(self,start_node,stop_node):
         threshold=heuristic_distance[start_node.value]

         while True:
            path=[]
            result,threshold=self.find_f_contour(start_node,0,threshold,stop_node.value,path)
            if result is True:
               return path
            if result == float('inf'):
               return None
     def find_f_contour(self,node,cost,threshold,goal,path):
         #print('threshold   is {} ,cost is {} node  is {}'.format(threshold,cost,node.value))
         if node.value not in path:
            path.append(node.value)

         f=cost+heuristic_distance[node.value]
         #print('f   is {} , node  is {}'.format(f,node.value))
         if f > threshold:
            return False,f
         if node.value == goal:
            return True,threshold
         min_cost=float('inf')
         neighbours=self.get_neighbors(node.value)
         for (m,weight) in neighbours:
            if m.value not in path:
                new_cost=cost+weight
                path.append(m.value)
                #print('neighbour is {} and path is {}'.format(m.value,path))
                result,threshold=self.find_f_contour(m,new_cost,threshold,goal,path)
                if result == True:
                    return True,threshold
                path.pop()
                if threshold < min_cost:
                    min_cost = threshold
         print('min_cost is {}'.format(min_cost))
         return False,min_cost




     def __init__(self):
        self.graph_str = {
                        'New Delhi': [('A', (5,64)), ('B', (2,60)), ('C', (7,82)),('Chennai', (10,85))],
                        'A': [('New Delhi',(5,64)), ('E', (1,110))],
                        'B': [('New Delhi',(2,60)), ('Chennai', (3,62))],
                        'C': [('New Delhi',(7,82)), ('D', (5,45))],
                        'D': [('C', (5,45)),  ('Chennai',(6,32))],
                        'E': [('A', (1,110)), ('Chennai',(4,55))],
                        'Chennai': [ ('B', (3,62)), ('D', (6,32)),('E', (4,55))]



                        }

        self.graph={}

graph=Graph()
graph.build_graph()
start_node=Node(0,'New Delhi')
stop_node=Node(0,'Chennai')
#path=graph.aStarSearch(start_node,stop_node)
#print('Path found: {}'.format(path))
print(graph.ida_star_search(start_node,stop_node))
print(graph.get_random_path(start_node,stop_node))
print(graph.get_weight('C','D'))


['New Delhi', 'B', 'Chennai']
['New Delhi', 'A', 'E', 'Chennai']
5


In [56]:
import random

class GeneticAlgorithm:
    def __init__(self, graph, start_node, stop_node, population_size=100, mutation_rate=0.01, generations=100):
        self.graph = graph
        self.start_node = start_node
        self.stop_node = stop_node
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations

    def generate_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.graph.get_random_path(start_node,stop_node)
            population.append(individual)
        return population
    def calculate_fitness(self,individual):
        cost=0
        for i in range(len(individual) - 1):
            cost+=self.graph.get_weight(individual[i],individual[i+1])
        return 1/cost

    def select_parents(self,population, fitness_scores):
        total_fitness = sum(fitness_scores)
        if total_fitness > 0:
            probabilities = [fitness / total_fitness for fitness in fitness_scores]
            return random.choices(population, weights=probabilities, k=2)
        return random.choices(population, k=2)
    def crossover(self,parent1, parent2):
        crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    def mutate(self,individual):
        if random.random() < self.mutation_rate:
            mutation_point = random.randint(0, len(individual) - 2)
            neighbours = self.graph.get_neighbors(individual[mutation_point])
            if neighbours:
                my_next_random_neighbour = random.choice(neighbours)[0]
                individual[mutation_point + 1] = my_next_random_neighbour.value
        return individual
    def execute(self):
        population = self.generate_population()

        for generation in range(self.generations):
            fitness_scores = [self.calculate_fitness(individual) for individual in population]

            new_population = []

            for _ in range(len(population) // 2):
                parent1, parent2 = self.select_parents(population, fitness_scores)
                child1, child2 = self.crossover(parent1, parent2)
                new_population.extend([self.mutate(child1), self.mutate(child2)])

            population = new_population

        # Display the best individual found
        best_individual = max(population, key=self.calculate_fitness)
        print("Best individual:", best_individual, "Fitness:", self.calculate_fitness(best_individual))


In [57]:
graph=Graph()
graph.build_graph()
start_node=Node(0,'New Delhi')
stop_node=Node(0,'Chennai')
ga=GeneticAlgorithm(graph,start_node,stop_node)
ga.execute()

Best individual: ['New Delhi', 'B', 'New Delhi'] Fitness: 0.25
